In [2]:
import numpy as np
import pandas as pd
import polars as pl
import sys
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px


pd.set_option('display.max_columns',None)
import psycopg2


#to scale the data using z-score 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#Algorithms to use
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

#Metrics to evaluate the model
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve

import warnings
warnings.filterwarnings("ignore")

#importing PCA and TSNE
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [53]:
vcf = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/vcf_files_all/KHGLBS622.xlsx')
vcf['SAS_Allele_Frequencies'] = vcf['SAS_Allele_Frequencies'].replace('', 0).astype(float)
vcf = vcf[vcf['SAS_Allele_Frequencies'] <= 0.6]
vcf

,Chr:Pos,Ref/Alt,Zygosity,Gene Name,Consequence,rsID,SAS_Allele_Frequencies,CLIN_SIG
3,1:946247,G/A,Homozygous,NOC2L,synonymous_variant,rs2272757,0.441893,Benign
4,1:946538,G/A,Heterozygous,NOC2L,missense_variant,rs35471880,0.029353,Benign
15,1:978953,C/G,Homozygous,PERM1,missense_variant,rs13302979,0.556310,Benign
16,1:979472,G/C,Homozygous,PERM1,missense_variant,rs13303368,0.512580,Benign
18,1:979560,T/C,Homozygous,PERM1,synonymous_variant,rs13303033,0.509585,Benign
...,...,...,...,...,...,...,...,...
26280,X:156005349,C/T,Heterozygous,IL9R,synonymous_variant,rs150608249,0.057708,Benign
26281,Y:3019783,A/C,Homozygous,LINC00278,intron_variant,rs9786184,0.269669,Benign
26282,Y:6885470,A/G,Homozygous,AMELY,intron_variant,rs9785971,0.270199,Benign
26283,Y:7000077,A/G,Homozygous,TBL1Y,intron_variant,rs1865680,0.239735,Benign


In [54]:
vcf['CHROM'] = 'chr' + vcf['Chr:Pos'].str.split(':').str[0]
vcf['POS'] = vcf['Chr:Pos'].str.split(':').str[1]
vcf['REF'] = vcf['Ref/Alt'].str.split('/').str[0]
vcf['ALT'] = vcf['Ref/Alt'].str.split('/').str[1]
vcf = vcf[['CHROM', 'POS','rsID', 'REF', 'ALT', 'Zygosity', 'Gene Name', 'Consequence', 'CLIN_SIG']]
vcf['consequence'] = vcf['Consequence'].str.split(',').str[0].str.replace('_', ' ')
vcf

,CHROM,POS,rsID,REF,ALT,Zygosity,Gene Name,Consequence,CLIN_SIG,consequence
3,chr1,946247,rs2272757,G,A,Homozygous,NOC2L,synonymous_variant,Benign,synonymous variant
4,chr1,946538,rs35471880,G,A,Heterozygous,NOC2L,missense_variant,Benign,missense variant
15,chr1,978953,rs13302979,C,G,Homozygous,PERM1,missense_variant,Benign,missense variant
16,chr1,979472,rs13303368,G,C,Homozygous,PERM1,missense_variant,Benign,missense variant
18,chr1,979560,rs13303033,T,C,Homozygous,PERM1,synonymous_variant,Benign,synonymous variant
...,...,...,...,...,...,...,...,...,...,...
26280,chrX,156005349,rs150608249,C,T,Heterozygous,IL9R,synonymous_variant,Benign,synonymous variant
26281,chrY,3019783,rs9786184,A,C,Homozygous,LINC00278,intron_variant,Benign,intron variant
26282,chrY,6885470,rs9785971,A,G,Homozygous,AMELY,intron_variant,Benign,intron variant
26283,chrY,7000077,rs1865680,A,G,Homozygous,TBL1Y,intron_variant,Benign,intron variant


In [55]:
df_1 = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Madhu_folder_04_07_2023/kidney_health_final.vcf/consequence.xlsx')
df_1

,consequence,Consequence_score
0,transcript ablation,10/10
1,splice acceptor variant,8/10
2,splice donor variant,8/10
3,stop gained,10/10
4,frameshift variant,10/10
5,stop lost,9/10
6,start lost,9/10
7,transcript amplification,8/10
8,inframe insertion,6/10
9,inframe deletion,6/10


In [56]:
merged_2 = pd.merge(vcf, df_1, on='consequence', how='left', sort=False)
#merged_1.to_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/data_287.xlsx', index=False)
merged_2

,CHROM,POS,rsID,REF,ALT,Zygosity,Gene Name,Consequence,CLIN_SIG,consequence,Consequence_score
0,chr1,946247,rs2272757,G,A,Homozygous,NOC2L,synonymous_variant,Benign,synonymous variant,3/10
1,chr1,946538,rs35471880,G,A,Heterozygous,NOC2L,missense_variant,Benign,missense variant,7/10
2,chr1,978953,rs13302979,C,G,Homozygous,PERM1,missense_variant,Benign,missense variant,7/10
3,chr1,979472,rs13303368,G,C,Homozygous,PERM1,missense_variant,Benign,missense variant,7/10
4,chr1,979560,rs13303033,T,C,Homozygous,PERM1,synonymous_variant,Benign,synonymous variant,3/10
...,...,...,...,...,...,...,...,...,...,...,...
14203,chrX,156005349,rs150608249,C,T,Heterozygous,IL9R,synonymous_variant,Benign,synonymous variant,3/10
14204,chrY,3019783,rs9786184,A,C,Homozygous,LINC00278,intron_variant,Benign,intron variant,2/10
14205,chrY,6885470,rs9785971,A,G,Homozygous,AMELY,intron_variant,Benign,intron variant,2/10
14206,chrY,7000077,rs1865680,A,G,Homozygous,TBL1Y,intron_variant,Benign,intron variant,2/10


In [57]:
merged_2.Consequence_score.value_counts(dropna=False)

3/10     6192
7/10     4703
2/10     1967
4/10      591
1/10      356
NaN       165
6/10      112
10/10     103
8/10       15
9/10        4
Name: Consequence_score, dtype: int64

In [58]:
merged_2 = merged_2.fillna('0/10').astype('str')
merged_2 = merged_2[merged_2['Consequence_score'].apply(lambda x: eval(x) >= 6/10)]
merged_2

,CHROM,POS,rsID,REF,ALT,Zygosity,Gene Name,Consequence,CLIN_SIG,consequence,Consequence_score
1,chr1,946538,rs35471880,G,A,Heterozygous,NOC2L,missense_variant,Benign,missense variant,7/10
2,chr1,978953,rs13302979,C,G,Homozygous,PERM1,missense_variant,Benign,missense variant,7/10
3,chr1,979472,rs13303368,G,C,Homozygous,PERM1,missense_variant,Benign,missense variant,7/10
5,chr1,999842,rs2298214,C,A,Homozygous,HES4,missense_variant,Benign,missense variant,7/10
9,chr1,1087202,rs371815645,C,A,Heterozygous,C1orf159,missense_variant,Benign,missense variant,7/10
...,...,...,...,...,...,...,...,...,...,...,...
14180,chrX,153770343,rs2266879,G,A,Heterozygous,PLXNB3,missense_variant,Benign,missense variant,7/10
14185,chrX,153909841,rs2070099,C,A,Homozygous,ARHGAP4,missense_variant,Benign,missense variant,7/10
14188,chrX,153954909,rs1051152,A,G,Homozygous,HCFC1,missense_variant,Benign,missense variant,7/10
14191,chrX,153982271,rs2266890,C,T,Homozygous,TMEM187,missense_variant,Benign,missense variant,7/10


In [59]:
df_gene = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/condition_specific/Condition_specific_all_lit_genes/Final_lit/last_conditions/genes/Conditions_final_genes.xlsx')

merged_2['Gene Match'] = 'No'
merged_2['Matched_Gene'] = ''
    
# Iterate through each gene in vcf['Gene']
for index, genes in merged_2['Gene Name'].iteritems():
    if isinstance(genes, str):
        gene_list = genes.split(',')
        for gene in gene_list:
            if gene in df_gene['Gene Name'].values:
                merged_2.at[index, 'Gene Match'] = 'Yes'
                merged_2.at[index, 'Matched_Gene'] = gene
                break
    
df_gene = df_gene.rename({'Gene Name':'Matched_Gene'}, axis=1)

#merged_2 = merged_2.drop(columns=['Gene Match'], axis=1)

merged_2 = pd.merge(merged_2, df_gene, on= 'Matched_Gene', how = 'left', sort = False)
merged_2['Condition'] = merged_2['Condition'].fillna('No')
merged_2['Headings'] = merged_2['Headings'].fillna('No')
merged_2['21_Conditions_list'] = merged_2['21_Conditions_list'].fillna('No')
merged_2['Gene_Score'] = merged_2['Gene_Score'].fillna('No')
merged_2

,CHROM,POS,rsID,REF,ALT,Zygosity,Gene Name,Consequence,CLIN_SIG,consequence,Consequence_score,Gene Match,Matched_Gene,Condition,Headings,21_Conditions_list,Gene_Score
0,chr1,946538,rs35471880,G,A,Heterozygous,NOC2L,missense_variant,Benign,missense variant,7/10,No,,No,No,No,No
1,chr1,978953,rs13302979,C,G,Homozygous,PERM1,missense_variant,Benign,missense variant,7/10,No,,No,No,No,No
2,chr1,979472,rs13303368,G,C,Homozygous,PERM1,missense_variant,Benign,missense variant,7/10,No,,No,No,No,No
3,chr1,999842,rs2298214,C,A,Homozygous,HES4,missense_variant,Benign,missense variant,7/10,No,,No,No,No,No
4,chr1,1087202,rs371815645,C,A,Heterozygous,C1orf159,missense_variant,Benign,missense variant,7/10,No,,No,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4932,chrX,153770343,rs2266879,G,A,Heterozygous,PLXNB3,missense_variant,Benign,missense variant,7/10,No,,No,No,No,No
4933,chrX,153909841,rs2070099,C,A,Homozygous,ARHGAP4,missense_variant,Benign,missense variant,7/10,No,,No,No,No,No
4934,chrX,153954909,rs1051152,A,G,Homozygous,HCFC1,missense_variant,Benign,missense variant,7/10,No,,No,No,No,No
4935,chrX,153982271,rs2266890,C,T,Homozygous,TMEM187,missense_variant,Benign,missense variant,7/10,No,,No,No,No,No


In [60]:
df_3 = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/condition_specific/Condition_specific_all_lit_genes/Final_lit/last_conditions/new_final_output_concatenated.xlsx')

merged_2['POS'] = merged_2['POS'].astype('int64')
df_3['POS'] = df_3['POS'].astype('int64')

merged_2 = merged_2.rename({'Matched_Gene':'Gene'}, axis=1)

merged_3 = pd.merge(merged_2, df_3, on=['CHROM', 'POS', 'REF', 'ALT'], how='left', sort=False)
merged_3['Literature'] = merged_3['Literature'].fillna('No')
merged_3

,CHROM,POS,rsID,REF,ALT,Zygosity,Gene Name,Consequence,CLIN_SIG,consequence,Consequence_score,Gene Match,Gene,Condition,Headings,21_Conditions_list,Gene_Score,gene,Literature
0,chr1,946538,rs35471880,G,A,Heterozygous,NOC2L,missense_variant,Benign,missense variant,7/10,No,,No,No,No,No,NaN,No
1,chr1,978953,rs13302979,C,G,Homozygous,PERM1,missense_variant,Benign,missense variant,7/10,No,,No,No,No,No,NaN,No
2,chr1,979472,rs13303368,G,C,Homozygous,PERM1,missense_variant,Benign,missense variant,7/10,No,,No,No,No,No,NaN,No
3,chr1,999842,rs2298214,C,A,Homozygous,HES4,missense_variant,Benign,missense variant,7/10,No,,No,No,No,No,NaN,No
4,chr1,1087202,rs371815645,C,A,Heterozygous,C1orf159,missense_variant,Benign,missense variant,7/10,No,,No,No,No,No,NaN,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4932,chrX,153770343,rs2266879,G,A,Heterozygous,PLXNB3,missense_variant,Benign,missense variant,7/10,No,,No,No,No,No,NaN,No
4933,chrX,153909841,rs2070099,C,A,Homozygous,ARHGAP4,missense_variant,Benign,missense variant,7/10,No,,No,No,No,No,NaN,No
4934,chrX,153954909,rs1051152,A,G,Homozygous,HCFC1,missense_variant,Benign,missense variant,7/10,No,,No,No,No,No,NaN,No
4935,chrX,153982271,rs2266890,C,T,Homozygous,TMEM187,missense_variant,Benign,missense variant,7/10,No,,No,No,No,No,NaN,No


In [61]:
condition_filter = merged_3[merged_3['21_Conditions_list'] != 'No']
condition_filter

,CHROM,POS,rsID,REF,ALT,Zygosity,Gene Name,Consequence,CLIN_SIG,consequence,Consequence_score,Gene Match,Gene,Condition,Headings,21_Conditions_list,Gene_Score,gene,Literature
123,chr1,20650507,rs1043424,A,C,Heterozygous,"PINK1,PINK1-AS",missense_variant,"Benign,Benign",missense variant,7/10,Yes,PINK1,Parkisons Disease,Parkisons Disease,Parkinsons,8.0,NaN,No
164,chr1,30874473,rs2282440,G,A,Heterozygous,SDC3,missense_variant,Benign,missense variant,7/10,Yes,SDC3,Obesity,Obesity,Obesity,No,NaN,No
169,chr1,33479392,rs41265901,A,G,Heterozygous,ZSCAN20,missense_variant,Benign,missense variant,7/10,Yes,ZSCAN20,Diabetic Neuropathy,Diabetic Nephropathy,Diabetes,4.0,NaN,No
242,chr1,150512511,rs13294,G,A,Heterozygous,ECM1,missense_variant,Benign,missense variant,7/10,Yes,ECM1,Gut Health,Gut Health,Gut_Health,6.0,NaN,No
257,chr1,152305958,rs61816760,T,G,Heterozygous,FLG,missense_variant,Benign,missense variant,7/10,Yes,FLG,Dry Skin/fragile skin,Skin Health,Skin_Health,8.0,NaN,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4841,chr22,43927042,rs2076213,T,G,Heterozygous,PNPLA3,missense_variant,Benign,missense variant,7/10,Yes,PNPLA3,Fatty liver,Fatty liver,Fatty_Liver,8.0,NaN,No
4842,chr22,43927090,rs2076212,G,T,Heterozygous,PNPLA3,missense_variant,Benign,missense variant,7/10,Yes,PNPLA3,Fatty liver,Fatty liver,Fatty_Liver,8.0,NaN,No
4863,chr22,50525807,rs11479,G,A,Heterozygous,"SCO2,TYMP",missense_variant,"Benign,Benign",missense variant,7/10,Yes,SCO2,Muscular_health,Muscular_health,Muscular_health,4.0,NaN,No
4867,chr22,50572947,rs8142477,G,C,Heterozygous,"CHKB-CPT1B,CPT1B",missense_variant,"Benign,Benign",missense variant,7/10,Yes,CHKB-CPT1B,Muscular_health,Muscular_health,Muscular_health,No,NaN,No


In [62]:
condition_filter = condition_filter.set_index(['Gene Name', 'Gene', 'Gene_Score', 'rsID', 'Literature', 'CHROM', 'POS', 'REF',
       'ALT', 'Zygosity', 'Consequence', 'Consequence_score', 'CLIN_SIG', 'Condition', 'Headings', '21_Conditions_list']).apply(lambda x: x.str.split('; ').explode()).reset_index()

# Reorder the DataFrame
columns_to_move = ['Condition', 'Headings', '21_Conditions_list']
condition_filter = condition_filter[columns_to_move + [col for col in condition_filter.columns if col not in columns_to_move]]
condition_filter

,Condition,Headings,21_Conditions_list,Gene Name,Gene,Gene_Score,rsID,Literature,CHROM,POS,REF,ALT,Zygosity,Consequence,Consequence_score,CLIN_SIG,consequence,Gene Match,gene
0,Parkisons Disease,Parkisons Disease,Parkinsons,"PINK1,PINK1-AS",PINK1,8.0,rs1043424,No,chr1,20650507,A,C,Heterozygous,missense_variant,7/10,"Benign,Benign",missense variant,Yes,NaN
1,Obesity,Obesity,Obesity,SDC3,SDC3,No,rs2282440,No,chr1,30874473,G,A,Heterozygous,missense_variant,7/10,Benign,missense variant,Yes,NaN
2,Diabetic Neuropathy,Diabetic Nephropathy,Diabetes,ZSCAN20,ZSCAN20,4.0,rs41265901,No,chr1,33479392,A,G,Heterozygous,missense_variant,7/10,Benign,missense variant,Yes,NaN
3,Gut Health,Gut Health,Gut_Health,ECM1,ECM1,6.0,rs13294,No,chr1,150512511,G,A,Heterozygous,missense_variant,7/10,Benign,missense variant,Yes,NaN
4,Dry Skin/fragile skin,Skin Health,Skin_Health,FLG,FLG,8.0,rs61816760,No,chr1,152305958,T,G,Heterozygous,missense_variant,7/10,Benign,missense variant,Yes,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,Fatty liver,Fatty liver,Fatty_Liver,PNPLA3,PNPLA3,8.0,rs2076213,No,chr22,43927042,T,G,Heterozygous,missense_variant,7/10,Benign,missense variant,Yes,NaN
219,Fatty liver,Fatty liver,Fatty_Liver,PNPLA3,PNPLA3,8.0,rs2076212,No,chr22,43927090,G,T,Heterozygous,missense_variant,7/10,Benign,missense variant,Yes,NaN
220,Muscular_health,Muscular_health,Muscular_health,"SCO2,TYMP",SCO2,4.0,rs11479,No,chr22,50525807,G,A,Heterozygous,missense_variant,7/10,"Benign,Benign",missense variant,Yes,NaN
221,Muscular_health,Muscular_health,Muscular_health,"CHKB-CPT1B,CPT1B",CHKB-CPT1B,No,rs8142477,No,chr22,50572947,G,C,Heterozygous,missense_variant,7/10,"Benign,Benign",missense variant,Yes,NaN


In [63]:
drop_duplicates_filter = condition_filter.copy()

In [64]:
# Sample list of condition names
conditions_list = [
    'Diabetes',
    'High_Blood_Pressure',
    'Cardiac_Health',
    'Cholesterol_Disorders',
    'Thyroid_Disorders',
    'Parkinsons',
    'Dementia',
    'Headaches',
    'Allergies',
    'Anemia',
    'Fatty_Liver',
    'Gall_stones',
    'Pancreatic_Disorders',
    'Gut_Health',
    'Gastritis',
    'Glomerular_Diseases',
    'Interstitial_Nephritis',
    'Renal_stones',
    'Skin_Health',
    'Osteoporosis',
    'Arthritis_Degenerative_Joint',
    'Muscular_Dystrophy',
    'Fatigue',
    'Mood_Disorders',
    'Obesity',
    'Seizures',
    'Bone_Joint_health',
    'Muscular_health'
]

# Create an Excel writer object
excel_writer = pd.ExcelWriter(r'C:/Users/GenepoweRx_Madhu/Documents/Processed_vcf_files/KHGLBS622_final_output.xlsx', engine='xlsxwriter')

# Define a workbook
workbook = excel_writer.book

# Define the green and pink cell formats
green_format = workbook.add_format({'bg_color': '#00FF00'})  # Green color
pink_format = workbook.add_format({'bg_color': '#FF00FF'})   # Pink color

# A flag to keep track of whether at least one group was found
found_at_least_one_group = False

# Iterate through each condition and save it as a separate sheet
for condition in conditions_list:
    group = drop_duplicates_filter[drop_duplicates_filter['21_Conditions_list'] == condition]
    if not group.empty:
        found_at_least_one_group = True
        # Modify the name to make it fit within the Excel sheet name limit (31 characters)
        sheet_name = re.sub(r'[\/:*?"<>|]', '_', condition)[:31]

        # Sort the group first by the "Condition" column and then by the "Literature" column
        group_sorted = group.sort_values(by=['Condition', 'Literature'], ascending=[True, False])

        # Delete the '21_Conditions_list' column
        group_sorted = group_sorted.drop(columns=['21_Conditions_list', 'Gene Name'])

        columns_to_keep = ['Condition', 'Headings', 'Gene', 'Gene_Score', 'Zygosity', 'CLIN_SIG','Consequence', 'Consequence_score', 'CHROM', 'POS','rsID','Literature', 'REF', 'ALT']

        # Select only the desired columns
        group_sorted = group_sorted[columns_to_keep]

        # Write each group to a separate sheet with the modified sheet name
        group_sorted.to_excel(excel_writer, sheet_name=sheet_name, index=False)

        # Access the worksheet to apply conditional formatting
        worksheet = excel_writer.sheets[sheet_name]

        # Apply conditional formatting based on 'Literature' column values
        for i, value in enumerate(group_sorted['Literature']):
            if value == 'Yes':
                worksheet.write(i + 1, columns_to_keep.index('Literature'), value, green_format)
            elif value == 'No':
                worksheet.write(i + 1, columns_to_keep.index('Literature'), value, pink_format)

# If no groups were found, create a sheet named "Condition not found" and write the message in cell A1
for condition in conditions_list:
    if condition not in excel_writer.sheets:
        # Modify the name to make it fit within the Excel sheet name limit (31 characters)
        sheet_name = re.sub(r'[\/:*?"<>|]', '_', condition)[:31]
        worksheet = excel_writer.book.add_worksheet(sheet_name)
        worksheet.write(0, 0, "Variants not found")

# Save the Excel file
excel_writer.save()

In [4]:
def read_bed_file(bed_file):
    bed_positions = set()
    with open(bed_file, 'r') as f:
        for line in f:
            if line.startswith('#'):  # Skip header lines if present
                continue
            fields = line.strip().split('\t')
            if len(fields) >= 3:
                chrom = fields[0]
                try:
                    start = int(fields[1])
                    end = int(fields[2])
                except ValueError:
                    continue  # Skip this line if start or end position is not an integer
                for pos in range(start, end + 1):
                    bed_positions.add((chrom, pos))
    return bed_positions

def normalize_chrom_name(chrom):
    return chrom.split('_')[0]

def filter_vcf_file(vcf_file, bed_positions):
    filtered_vcf_records = []
    with open(vcf_file, 'r') as f:
        for line in f:
            if line.startswith('#'):  # Preserve header lines in the output
                filtered_vcf_records.append(line)
                continue
            fields = line.strip().split('\t')
            if len(fields) >= 2:
                raw_chrom = fields[0]
                chrom = normalize_chrom_name(raw_chrom)
                try:
                    pos = int(fields[1])
                except ValueError:
                    continue  # Skip this line if 'POS' is not an integer
                if (chrom, pos) in bed_positions:
                    filtered_vcf_records.append(line)
    return filtered_vcf_records

def write_filtered_vcf(filtered_vcf_records, output_file):
    with open(output_file, 'w') as f:
        for record in filtered_vcf_records:
            f.write(record)

def main():
    bed_file = r'C:/Users/GenepoweRx_Madhu/Downloads/BED_files/kalyani_mam_covered.bed'
    vcf_file = r'C:/Users/GenepoweRx_Madhu/Downloads/vcf_files_all/KHAIGHGPTTL287-WESPOOL160_S70_L003_R.vcf'
    output_file = r'C:/Users/GenepoweRx_Madhu/Downloads/COVERED_VCF_FILES_BED/KHAIGHGPTTL287-WESPOOL160_S70_L003_R.vcf'

    bed_positions = read_bed_file(bed_file)
    filtered_vcf_records = filter_vcf_file(vcf_file, bed_positions)
    write_filtered_vcf(filtered_vcf_records, output_file)

if __name__ == "__main__":
    main()

In [12]:
vcf = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/COVERED_VCF_FILES_BED/KHAIGHGPTTL287-WESPOOL160_S70_L003_R.vcf', comment= '#', sep = '\t', header=None, low_memory=False)
vcf.columns = ['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMPLE']

vcf['GT'] = vcf['SAMPLE'].str.split(':').str[0]
vcf['AD'] = vcf['SAMPLE'].str.split(':').str[1]
vcf['DP'] = vcf['SAMPLE'].str.split(':').str[2]
vcf['GQ'] = vcf['SAMPLE'].str.split(':').str[3]
vcf['PGT'] = vcf['SAMPLE'].str.split(':').str[4]
vcf['PID'] = vcf['SAMPLE'].str.split(':').str[5]
vcf['PL'] = vcf['SAMPLE'].str.split(':').str[6]

vcf = vcf[['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'GT', 'AD', 'DP', 'GQ', 'PGT', 'PID', 'PL']]
vcf

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,INFO,GT,AD,DP,GQ,PGT,PID,PL
0,chr1,69270,.,A,G,67.28,.,AC=2;AF=1;AN=2;DP=3;ExcessHet=3.0103;FS=0.000;...,1/1,"0,3",3,9,"95,9,0",NaN,NaN
1,chr1,69511,.,A,G,1964.77,.,AC=2;AF=1;AN=2;DP=68;ExcessHet=3.0103;FS=0.000...,1/1,"0,68",68,99,"1993,204,0",NaN,NaN
2,chr1,69897,.,T,C,31.74,.,AC=2;AF=1;AN=2;DP=2;ExcessHet=3.0103;FS=0.000;...,1/1,"0,2",2,6,"59,6,0",NaN,NaN
3,chr1,931131,.,C,CCCCT,1090.73,.,AC=2;AF=1;AN=2;BaseQRankSum=-0.671;ClippingRan...,1/1,"1,27",28,58,"1128,58,0",NaN,NaN
4,chr1,941119,.,A,G,1290.77,.,AC=2;AF=1;AN=2;DP=38;ExcessHet=3.0103;FS=0.000...,1/1,"0,38",38,99,"1319,114,0",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48809,chr9_KI270719v1_random,154634,.,G,T,268.77,.,AC=1;AF=0.5;AN=2;BaseQRankSum=-2.188;ClippingR...,0/1,"60,11",71,99,0|1,154555_G_A,"297,0,2794"
48810,chr9_KI270719v1_random,154676,.,G,T,156.77,.,AC=1;AF=0.5;AN=2;BaseQRankSum=-0.329;ClippingR...,0/1,"61,8",69,99,0|1,154555_G_A,"185,0,2630"
48811,chr9_KI270719v1_random,154703,.,C,T,44.77,.,AC=1;AF=0.5;AN=2;BaseQRankSum=1.987;ClippingRa...,0/1,"69,7",76,73,0|1,154555_G_A,"73,0,2715"
48812,chr16_KI270728v1_random,1459097,.,G,A,63.28,.,AC=2;AF=1;AN=2;DP=3;ExcessHet=3.0103;FS=0.000;...,1/1,"0,3",3,9,"91,9,0",NaN,NaN


In [8]:
vcf = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/vcf_files_all/KHAIGHGPTTL287-WESPOOL160_S70_L003_R.vcf', comment= '#', sep = '\t', header=None, low_memory=False)
vcf.columns = ['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMPLE']
vcf

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SAMPLE
0,chr1,13273,.,G,C,43.74,.,AC=2;AF=1;AN=2;DP=2;ExcessHet=3.0103;FS=0.000;...,GT:AD:DP:GQ:PL,"1/1:0,2:2:6:71,6,0"
1,chr1,13417,.,C,CGAGA,166.73,.,AC=1;AF=0.5;AN=2;BaseQRankSum=-0.967;ClippingR...,GT:AD:DP:GQ:PL,"0/1:1,5:6:40:204,0,40"
2,chr1,14522,.,G,A,91.77,.,AC=1;AF=0.5;AN=2;BaseQRankSum=-0.000;ClippingR...,GT:AD:DP:GQ:PGT:PID:PL,"0/1:2,3:5:75:0|1:14522_G_A:120,0,75"
3,chr1,14542,.,A,G,91.77,.,AC=1;AF=0.5;AN=2;BaseQRankSum=-0.000;ClippingR...,GT:AD:DP:GQ:PGT:PID:PL,"0/1:2,3:5:75:0|1:14522_G_A:120,0,75"
4,chr1,14574,.,A,G,133.77,.,AC=1;AF=0.5;AN=2;BaseQRankSum=-0.000;ClippingR...,GT:AD:DP:GQ:PGT:PID:PL,"0/1:2,4:6:99:0|1:14522_G_A:162,0,102"
...,...,...,...,...,...,...,...,...,...,...
135350,chrUn_JTFH01000319v1_decoy,1429,.,G,T,21.77,LowQual,AC=2;AF=1;AN=2;DP=7;ExcessHet=3.0103;FS=0.000;...,GT:AD:DP:GQ:PL,"1/1:0,2:2:6:49,6,0"
135351,chrUn_JTFH01000981v1_decoy,2235,.,T,C,15.65,LowQual,AC=2;AF=1;AN=2;DP=8;ExcessHet=3.0103;FS=0.000;...,GT:AD:DP:GQ:PL,"1/1:0,1:1:3:42,3,0"
135352,chrUn_JTFH01000981v1_decoy,2372,.,A,C,15.65,LowQual,AC=2;AF=1;AN=2;DP=4;ExcessHet=3.0103;FS=0.000;...,GT:AD:DP:GQ:PL,"1/1:0,1:1:3:42,3,0"
135353,chrUn_JTFH01001016v1_decoy,1437,.,G,C,107.28,.,AC=2;AF=1;AN=2;DP=3;ExcessHet=3.0103;FS=0.000;...,GT:AD:DP:GQ:PGT:PID:PL,"1/1:0,3:3:9:0|1:1437_G_C:135,9,0"


In [17]:
x = vcf[vcf['POS'] == 69270]
x.INFO.iloc[0]

'AC=2;AF=1;AN=2;DP=3;ExcessHet=3.0103;FS=0.000;MLEAC=2;MLEAF=1;MQ=27.15;QD=22.43;SOR=2.833'

In [65]:
import pandas as pd

# Assuming you have two dataframes: df1 and df2
# Replace 'haplotype' with the actual column name you want to map

# Example data
data1 = {'rsID': ['rs1', 'rs2,rs3', 'rs4,rs5'], 'haplotype': ['A', 'B', 'C']}
data2 = {'rsID': ['rs3', 'rs1', 'rs5,rs4'], 'haplotype': ['X', 'Y', 'Z']}

df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)
df1

,rsID,haplotype
0,rs1,A
1,"rs2,rs3",B
2,"rs4,rs5",C


In [69]:
import pandas as pd

# Assuming 'data' is the column containing the comma-separated values
data = ['rs1799930,rs1041983', 'rs1799930,rs1041983', 'rs2279343,rs3745274']

# Convert the data column to a DataFrame
df = pd.DataFrame({'data': data})
df

,data
0,"rs1799930,rs1041983"
1,"rs1799930,rs1041983"
2,"rs2279343,rs3745274"


In [70]:
# Split each row into a list of values, sort the values, and join them back into a string
df['data'] = df['data'].apply(lambda x: ','.join(sorted(x.split(','))))

# Print the sorted DataFrame
df

,data
0,"rs1041983,rs1799930"
1,"rs1041983,rs1799930"
2,"rs2279343,rs3745274"
